# SABR MODEL CALIBRATION & MARKET SMILE VIEWER TOOL
### Historical CSV Calibration + Live Fetch (optional)


# 🧮 SABR Volatility Smile Calibration Dashboard

Analyze, calibrate, and visualize **market-implied volatility smiles**  
for major equity indices (**FTSE 100**, **S&P 500**, **EURO STOXX 50**).  
Use **historical CSV data** or **live Yahoo Finance** feeds to compare  
**observed** versus **model-fitted** volatility curves.

---

## 📖 Overview

This project implements a full **SABR model calibration and volatility-smile analysis** framework.  
It estimates the SABR parameters α (initial vol), ρ (correlation), ν (vol-of-vol) for a fixed β,  
and plots market vs model implied volatilities across strikes or moneyness (K/F).

**Features**
- Historical calibration from **CSV market data**  
- Live calibration via **Yahoo Finance**

---

## 📂 Project Structure

| File / Folder | Purpose |
|:--|:--|
| `FTSE_SABR_Multi.ipynb` | Interactive notebook dashboard |
| `sabr_pipeline.py` | Core SABR workflow: load → calibrate → plot |
| `sabr_utils.py` | Mathematical utilities (Hagan implied vol etc.) |
| `sabr_data.py` | CSV I/O and time-to-maturity helpers |
| `sabr_yahoo.py` | Live option-smile fetcher (Yahoo Finance) |
| `MarketData/` | Historical / fetched option-smile data |
| `requirements.txt` | Python dependencies |

---

## The SABR Model (Hagan et al., 2002)

The **SABR (Stochastic Alpha Beta Rho)** model describes the forward price and its stochastic volatility:

$$
dF_t = \sigma_t F_t^{\beta} dW_t
$$

$$
d\sigma_t = \nu\,\sigma_t\,dZ_t, \qquad dW_t\,dZ_t = \rho\,dt
$$

**Parameters**

- α = σ₀   — initial volatility level  
- β ∈ [0, 1]  — elasticity of volatility w.r.t Fₜ  
- ρ             — correlation between price & volatility  
- ν             — volatility of volatility  

---

## 🧮 Hagan’s Implied-Volatility Approximation

For strike K, forward F, and maturity T, the Black–Scholes implied volatility under SABR is:

$$
\sigma_{BS}(F,K) =
\frac{\alpha}{(F K)^{(1-\beta)/2}}
\frac{z}{x(z)}
\Bigg[
1 +
\Big(
\frac{(1-\beta)^2}{24}\frac{\alpha^2}{(F K)^{1-\beta}}
+ \frac{\rho\,\beta\,\nu\,\alpha}{4 (F K)^{(1-\beta)/2}}
+ \frac{(2-3\rho^2)\nu^2}{24}
\Big)T
\Bigg].
$$

where

$$
z = \frac{\nu}{\alpha}(F K)^{(1-\beta)/2}\ln\!\Big(\frac{F}{K}\Big),
\qquad
x(z) = \ln\!\Bigg(\frac{\sqrt{1-2\rho z + z^2}+z-\rho}{1-\rho}\Bigg).
$$

**ATM limit (F = K):**

$$
\sigma_{BS}(F,F) =
\frac{\alpha}{F^{1-\beta}}
\Bigg[
1 +
\Big(
\frac{(1-\beta)^2}{24}\frac{\alpha^2}{F^{2(1-\beta)}}
+ \frac{\rho\,\beta\,\nu\,\alpha}{4 F^{1-\beta}}
+ \frac{(2-3\rho^2)\nu^2}{24}
\Big)T
\Bigg].
$$

---

## 📊 Outputs

Each calibration produces:

- Fitted parameters (α, ρ, ν)  
- Objective function (SSE)  
- Plots:  
    • Market vs SABR Fit (by Strike)  
    • Market vs SABR Fit (by Moneyness K/F)  

---

## ⚙️ Environment Setup

```bash
# create and activate env
python -m venv sabr_model
sabr_model\Scripts\activate        # Windows
# or
source sabr_model/bin/activate     # macOS / Linux

# install dependencies
pip install -r requirements.txt


📚 Reference

Hagan, P. S., Kumar, D., Lesniewski, A. S., & Woodward, D. E. (2002).
Managing Smile Risk. Wilmott Magazine, September 2002.


© 2025 Quantitative Finance Engineering Toolkit — for academic & professional use.

## INTERACTIVE DASHBOARD 

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML
import ipywidgets as w
from pathlib import Path

import importlib
import sabr_pipeline
importlib.reload(sabr_pipeline)
from sabr_pipeline import (
    fetch_and_save, calibrate_sabr, compute_model_vols, plot_smile,
    plot_parameter_sensitivity, CSV_PATHS, load_csv_smile
)

# --- Dark Theme Styling ---
HTML("""
<style>
.jp-Notebook, .jp-NotebookPanel, .jp-RenderedHTMLCommon {
  background-color: #121212 !important;
  color: #e0e0e0 !important;
}
.widget-button {
  background-color: #007acc !important;
  color: white !important;
  border-radius: 6px;
}
h1, h2, h3, strong { color: #ffb703 !important; }
</style>
""")

# --- UI Widgets ---
index_dd = w.Dropdown(
    options=['FTSE100', 'S&P500', 'EUROSTOXX50'],
    value='FTSE100',
    description='Index'
)
source_dd = w.Dropdown(
    options=['csv', 'yahoo'],
    value='csv',
    description='Source'
)
beta_slider = w.FloatSlider(value=0.8, min=0.0, max=1.0, step=0.05, description='beta (β)')
days_slider = w.IntSlider(value=90, min=7, max=365, step=7, description='Target Days')
btn = w.Button(description='Run Calibration', button_style='primary')
out = w.Output()

# --- Ensure Results folder exists ---
Path("Results").mkdir(exist_ok=True)

# --- Main Function ---
def run(_=None):
    out.clear_output()
    idx = index_dd.value
    beta = float(beta_slider.value)

    try:
        # ======================================================
        # CSV MODE
        # ======================================================
        if source_dd.value == 'csv':
            df, meta = load_csv_smile(idx)
            asof = pd.to_datetime(df['AsOf'].iloc[0])
            expiry = pd.to_datetime(df['Expiry'].iloc[0])
            T = (expiry - asof).days / 365.0
            F = float(meta.get('Forward', df['Strike'].median()))
            strikes = df['Strike'].astype(float).values
            market = df['MarketVol'].astype(float).values

            print(f"AsOf={meta['AsOf']}  Expiry={meta['Expiry']}  T={T:.6f}y  F={F:.6f}")
            print("vol units check (median):", float(np.nanmedian(market)))

            calib = calibrate_sabr(F, T, strikes, market, beta=beta)
            model = compute_model_vols(F, T, calib['alpha'], beta, calib['rho'], calib['nu'], strikes)

            with out:
                display(Markdown(
                    f"**CSV loaded:** `{CSV_PATHS[idx]}`\n\n"
                    f"- AsOf: **{asof.date()}**, Expiry: **{expiry.date()}**,  T≈**{T:.4f}**\n"
                    f"- Forward: **{F:.6f}**  β (fixed): **{beta:.2f}**"
                ))
                display(Markdown(
                    f"### Calibrated Parameters\n"
                    f"- α: **{calib['alpha']:.6f}**, ρ: **{calib['rho']:.6f}**, ν: **{calib['nu']:.6f}**\n"
                    f"- SSE: **{calib['cost']:.6g}**"
                ))

                # --- Plots first ---
                plot_smile(strikes, market, model, title=f"{idx}: Market vs SABR Fit (β={beta:.2f})")
                plot_parameter_sensitivity(F, T, strikes, calib, beta, market)

                mny = strikes / float(F)
                plt.figure()
                plt.scatter(mny, market, s=18, label='Market')
                plt.plot(mny, model, label='SABR Fit', color='orange')
                plt.xlabel('Moneyness (K/F)')
                plt.ylabel('Implied Volatility')
                plt.title(f'{idx}: Market vs SABR Fit vs Moneyness (β={beta:.2f})')
                plt.legend()
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.show()

                # --- Volatility Comparison Table (AFTER plots) ---
                vol_table = pd.DataFrame({
                    "Strike": strikes,
                    "Moneyness (K/F)": strikes / F,
                    "Market Vol (%)": market * 100,
                    "SABR Implied Vol (%)": model * 100,
                    "Vol Diff (bps)": (model - market) * 10000
                })

                save_path = Path("Results") / f"{idx}_SABR_vol_table.csv"
                vol_table.to_csv(save_path, index=False)

                display(Markdown(
                    f"### 📄 Volatility Comparison Table (preview, full file saved to `Results/{idx}_SABR_vol_table.csv`)"
                ))
                display(
                    vol_table.head(10).style.format({
                        "Market Vol (%)": "{:.2f}",
                        "SABR Implied Vol (%)": "{:.2f}",
                        "Vol Diff (bps)": "{:+.1f}"
                    }).set_table_styles([
                        {'selector': 'th', 'props': [('background-color', '#2c2c2c'), ('color', 'white')]},
                        {'selector': 'td', 'props': [('background-color', '#1e1e1e'), ('color', '#e0e0e0')]}
                    ])
                )
                print(f"Full volatility table saved to {save_path}")

        # ======================================================
        # YAHOO MODE
        # ======================================================
        else:
            df, meta_fetch, canonical, dated = fetch_and_save(idx, target_days=int(days_slider.value))
            asof, expiry = pd.to_datetime(df['AsOf'].iloc[0]), pd.to_datetime(df['Expiry'].iloc[0])
            T = (expiry - asof).days / 365.0
            F = float(meta_fetch.get('Forward')) if meta_fetch.get('Forward') is not None else float(df['Strike'].median())
            strikes = df['Strike'].astype(float).values
            market = df['MarketVol'].astype(float).values

            print(f"AsOf={asof.date()}  Expiry={expiry.date()}  T={T:.6f}y  F={F:.6f}")
            print("vol units check (median):", float(np.nanmedian(market)))

            calib = calibrate_sabr(F, T, strikes, market, beta=beta)
            model = compute_model_vols(F, T, calib['alpha'], beta, calib['rho'], calib['nu'], strikes)

            with out:
                display(Markdown(
                    f"**Saved CSVs:**\n"
                    f"- Canonical: `{canonical}`\n"
                    f"- Snapshot: `{dated}`"
                ))
                display(Markdown(
                    f"### Inputs\n"
                    f"- Index: **{idx}**\n"
                    f"- AsOf: **{asof.date()}**, Expiry: **{expiry.date()}**,  T≈**{T:.4f}**\n"
                    f"- Forward: **{F:.6f}**  β (fixed): **{beta:.2f}**"
                ))
                display(Markdown(
                    f"### Calibrated Parameters\n"
                    f"- α: **{calib['alpha']:.6f}**, ρ: **{calib['rho']:.6f}**, ν: **{calib['nu']:.6f}**\n"
                    f"- SSE: **{calib['cost']:.6g}**"
                ))

                # --- Plots first ---
                plot_smile(strikes, market, model, title=f"{idx}: Market vs SABR Fit (β={beta:.2f})")

                mny = strikes / float(F)
                plt.figure()
                plt.scatter(mny, market, s=18, label='Market')
                plt.plot(mny, model, label='SABR Fit', color='orange')
                plt.xlabel('Moneyness (K/F)')
                plt.ylabel('Implied Volatility')
                plt.title(f'{idx}: Market vs SABR Fit vs Moneyness (β={beta:.2f})')
                plt.legend()
                plt.grid(True, linestyle='--', alpha=0.5)
                plt.show()

                # --- Volatility Comparison Table (AFTER plots) ---
                vol_table = pd.DataFrame({
                    "Strike": strikes,
                    "Moneyness (K/F)": strikes / F,
                    "Market Vol (%)": market * 100,
                    "SABR Implied Vol (%)": model * 100,
                    "Vol Diff (bps)": (model - market) * 10000
                })

                save_path = Path("Results") / f"{idx}_SABR_vol_table.csv"
                vol_table.to_csv(save_path, index=False)

                display(Markdown(
                    f"### 📄 Volatility Comparison Table (preview, full file saved to `Results/{idx}_SABR_vol_table.csv`)"
                ))
                display(
                    vol_table.head(10).style.format({
                        "Market Vol (%)": "{:.2f}",
                        "SABR Implied Vol (%)": "{:.2f}",
                        "Vol Diff (bps)": "{:+.1f}"
                    }).set_table_styles([
                        {'selector': 'th', 'props': [('background-color', '#2c2c2c'), ('color', 'white')]},
                        {'selector': 'td', 'props': [('background-color', '#1e1e1e'), ('color', '#e0e0e0')]}
                    ])
                )
                print(f"Full volatility table saved to {save_path}")

    except Exception as e:
        with out:
            display(Markdown(f"**Error:** {e}"))

# --- Display UI ---
btn.on_click(run)
display(w.VBox([w.HBox([index_dd, source_dd, beta_slider, days_slider]), btn, out]))
